In [1]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
from eth_keys import keys
from pymerkle import InmemoryTree as MerkleTree
from eth_utils import keccak
import random

# Utility function to check if a number is a power of two
def is_power_of_two(n):
    return n > 0 and (n & (n - 1)) == 0

# Function to compute SHA-256 hash of an object
def compute_sha256(message):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(message)

    return keccak(message)  # Ethereum uses Keccak-256 for hashing the message

    return digest.finalize()

# Function to verify file hash against the pre-committed value
def verify_file_hash(file_path, expected_hash):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    with open(file_path, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b""):
            digest.update(chunk)
    file_hash = digest.finalize().hex()
    return file_hash == expected_hash

# Function to verify hash of an object (pre_object or proof)
def verify_committed_hash(object_to_hash, committed_hash):
    computed_hash = compute_sha256(str(object_to_hash).encode()).hex()
    return computed_hash == committed_hash

# Function to verify signature using public key
def verify_signature_hex(public_key_hex, message_hash, signature_hex):
    public_key_bytes = bytes.fromhex(public_key_hex[2:])
    public_key = keys.PublicKey(public_key_bytes)
    signature_bytes = bytes.fromhex(signature_hex[2:])
    signature = keys.Signature(signature_bytes)
    return public_key.verify_msg_hash(message_hash, signature)

# Function to process file and verify Merkle Tree roots
def process_file_and_verify_roots(file_path, proof_roots, skip=10, rounding=5, full_verification=False):
    tree = MerkleTree(hash_type='sha256')
    level = 1
    
    with open(file_path, 'r') as f:
        for i, line in enumerate(f, start=1):
            if i % skip == 0:
                rounded_numbers = [round(float(num), rounding) for num in line.strip().split(',')]
                tree.append_entry(str(rounded_numbers).encode())
                
                if is_power_of_two(tree.get_size()):
                    root_hash = tree.get_state().hex()
                    if root_hash in proof_roots:
                        print(f'Verification passed at level {level}')
                        level += 1
                    else:
                        print('Verification failed')
    
    if full_verification and not is_power_of_two(tree.get_size()):
        root_hash = tree.get_state().hex()
        if root_hash in proof_roots:
            print('Full verification passed')
        else:
            print('Verification failed')




In [2]:
# Gather pre-committed data
pre_hash = 'fa2a1cd8684f118211889aeb7306f3444d7515b5ca409976e15c784a6e2cb170'
pre_object = {'code_hash': '000000001', 'file_hash': '218a6313c7d6ae484734a20395bd39efac1bb7fbd84f690afe94f77924649cae'}
public_key_hex = '0xf47e6e2ee26af422643bed0c7f4eb96b9de00be9ab6a3e0ab9aca07338dd28f90c78d3c170c7fe2c06171ddf67747bc181625d1f3926cab8cbfc7794bab23b43'
signature_hex = '0x720890f2a5e67409d96edf6b153568b97d3dad10324992f4537134e8087dd8a16802954c31ae401bef3819fe71a25b20a674e6dfbe67a4ca983f3dbd7a6d1d5600'

# Verify file hash against pre-committed value
file_path = "input.txt"
if verify_file_hash(file_path, pre_object['file_hash']):
    print('Input verified')
else:
    print('Input could not be verified!')

print(f"Please verify the code fingerprint {pre_object['code_hash']} against the used code")

# Verify the committed pre_object hash
if verify_committed_hash(pre_object, pre_hash):
    print('Pre hash verified')
else:
    print('Committed hash not verified!')

# Verify the signature on the pre_object
pre_object_hash = compute_sha256(str(pre_object).encode())
if verify_signature_hex(public_key_hex, pre_object_hash, signature_hex):
    print('Signature valid')
else:
    print('Signature invalid!')



Input verified
Please verify the code fingerprint 000000001 against the used code
Pre hash verified
Signature valid


In [3]:
# Import main hash and signature for proof object
main_hash = '1d03a59045b126fe2e580f75a30298d73a2f5204c15828fee375ec992c3f77c2'
proof_signature_hex = '0x5b5983f890ab3c714b3dfdd57cb7c841fc0793bb61459a0300893b73b120d41517897bb0aa5a72d4a2b0be87a9c121b94d2ccc8134cf2b64f98bc9d393ba70b300'
proof =  {'roots': ['9aebface6387aeff2b3b4a9c0ccf94b617c41587a699d8b568ad7173806f28d9', '9ff5cf6b07c47ca91ea47969f88fdb4aec17760c74fecc89503fd4e25ae6d420', '169ed591e4712c61bdf73184d29ff53981c4832a2bc75bdbac5ca85de16f6776', '39f2cf5a5923eb790af160aef3c16adb32471bc27fbbaad78461498bc1fab792', '4088f92bff4d709e5ba0c3cad88ddc61cf9ddcfaab3f657b995690131c617560'], 'data_hash': '00000001', 'seed': 42}

print(f"Please verify the data fingerprint {proof['data_hash']} against the used data")
print(f"Verify that this matches with the commited pre_hash {proof['pre_hash']}")
print(f"Make sure to use seed {proof['seed']}")

# Verify main proof hash
if verify_committed_hash(proof, main_hash):
    print('Proof hash verified')
else:
    print('Proof hash not verified!')

# Verify the signature on the proof object
proof_hash = compute_sha256(str(proof).encode())
if verify_signature_hex(public_key_hex, proof_hash, proof_signature_hex):
    print('Signature valid')
else:
    print('Signature invalid!')

# Verify the data using Merkle Tree and the proof object
process_file_and_verify_roots("testchain.txt", proof['roots'], skip=10, rounding=5)

Please verify the data fingerprint 00000001 against the used data


KeyError: 'pre_hash'